# Trace with LangChain

## Quick start

In [5]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

When we are using Ollama, logging a trace is optional. 

In [6]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user's request only based on the given context."),
    ("user", "Question: {question}\nContext: {context}")
])

model = ChatOllama(model='llama3.1')
output_parser = StrOutputParser()

chain = prompt | model | output_parser

"What a remarkable day it has been! This morning, leaders from around the globe gathered in a historic summit to address and resolve all existing world conflicts. After intense discussions and constructive negotiations, they were able to reach a comprehensive agreement that brought peace and harmony to every nation.\n\nThe meeting was attended by top diplomats, representatives from international organizations, and key stakeholders from over 100 countries. The atmosphere was collaborative and solution-focused, with a shared commitment to creating a more peaceful and prosperous world.\n\nKey highlights from the morning's meetings include:\n\n* A unified declaration against war and violence\n* Establishment of a new global framework for conflict resolution\n* Agreement on economic cooperation and trade partnerships\n* Commitment to environmental sustainability and climate action\n\nThe summit was a resounding success, marking a significant turning point in human history. As we move forwar

There're 2 ways.
1. Manually passing in a `LangChainTracer`
2. Using the `tracing_v2_enabled`

In [7]:
from langchain.callbacks.tracers import LangChainTracer

# 1. Instance as a callback
tracer = LangChainTracer()
chain.invoke({"question": "Am I using a callback?", "context": "I'm using a callback"}, config={"callbacks": [tracer]})

# 2. Context manager
from langchain_core.tracers.context import tracing_v2_enabled
with tracing_v2_enabled():
    chain.invoke({"question": "Am I using a context manager?", "context": "I'm using a context manager"})

# This will NOT be traced
chain.invoke({"question": "Am I being traced?", "context": "I'm not being traced"})

"No, you're not being traced."

## Log to a specific project

In [10]:
from langchain.callbacks.tracers import LangChainTracer

tracer = LangChainTracer(project_name="LangGraph Tutorial")
chain.invoke({"question": "Am I using a callback?", "context": "I'm using a callback"}, config={"callbacks": [tracer]})

from langchain_core.tracers.context import tracing_v2_enabled
with tracing_v2_enabled(project_name="LangGraph Tutorial"):
    chain.invoke({"question": "Am I using a context manager?", "context": "I'm using a context manager"})
